# 复赛数据
在给出若干日内来自某成熟国家xx的部分用户的点击购买数据，以及来自某待成熟国家yy和待成熟国家zz的A部分用户的点击购买数据，以及国家yy和zz的B部分用户的截止最后一条购买数据之前的所有点击购买数据，让参赛人预测B部分用户的最后一条购买数据。

商品属性表: 商品的类目id、店铺id以及加密价格，其中价格的加密函数f(x)为一个单调增函数。
训练数据: 给出xx国的用户的点击、购买数据和yy国、zz国的A部分用户的点击、购买数据。
测试数据: 给出yy国、zz国的B部分用户的最后一条购买数据之前的点击购买数据.

训练数据和测试数据的数据结构是一样的，其中各字段含义如下：
country_id: 买家国家id, 只有'xx','yy','zz'三种取值
buyer_admin_id: 买家id
item_id: 商品id
log_time: 商品详情页访问时间
irank: 每个买家对应的所有记录按照时间顺序的逆排序
buy_flag: 当日是否购买

数据集特点：
每个用户有若干条点击数据和至少1条购买数据 （但测试数据中该条购买记录可能未给出到选手）
每个用户的最后一条数据的buy_flag一定为1 （但测试数据中该条数据未给出到选手）
测试数据中每个用户的最后一条点击数据（也是购买数据）所对应的商品一定在训练数据中出现过.
可能存在少量跨国买家.

要求选手提交的数据
关于yy国、zz国的B部分用户每个用户的最后一条购买数据的预测Top30

提交说明：
选手提交的CSV文件, 其格式应如下：
buyer_admin_id,predict 1,predict 2,…,predict 30
其中buyer_admin_id为买家id, predict 1 ,…, predict 30 为预测用户购买商品Top30的item_id依概率从高到低排序，不含表头，例如：

1233434,4354,23432,6546,...,91343

2132133,154,20987,34349,...,78772



评估方法：
MRR(Mean Reciprocal Rank)： 首先对选手提交的表格中的每个用户计算用户得分
$$
score\left(buyer\right) = \sum_{k=1}^{30}\frac{s\left(buyer,k\right)}{k}
$$

其中, 如果选手对该buyer的预测结果predict k命中该buyer的最后一条购买数据则$s\left(buyer,k\right)=1$; 否则$s\left(buyer,k\right)=0$. 最终得分为所有这些buyer的平均值。


In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
item = pd.read_csv('../data/Antai_AE/Antai_AE_round2_item_attr_20190813.csv')
train = pd.read_csv('../data/Antai_AE/Antai_AE_round2_train_20190813.csv')
test = pd.read_csv('../data/Antai_AE/Antai_AE_round2_test_20190813.csv')

In [4]:
item.isnull().any()

item_id       False
cate_id       False
store_id      False
item_price    False
dtype: bool

In [5]:
train.head()

,country_id,buyer_admin_id,item_id,log_time,irank,buy_flag
0,xx,1,7554,2018-04-19 10:59:56,79,0
1,xx,1,7937,2018-04-19 11:17:45,62,0
2,xx,1,7544,2018-04-18 22:19:30,81,0
3,xx,1,7559,2018-04-18 22:15:27,82,0
4,xx,1,7554,2018-04-19 11:28:34,46,0


In [6]:
test.head()

,country_id,buyer_admin_id,item_id,log_time,irank,buy_flag
0,zz,186,5759164,2018-04-16 05:11:47,37,0
1,zz,186,2321601,2018-04-16 04:57:35,48,0
2,zz,186,5244747,2018-04-17 12:24:43,2,0
3,zz,186,2136020,2018-04-17 11:53:41,8,0
4,zz,186,2137602,2018-04-16 04:57:36,47,0


In [8]:
train["log_time"].min(), train["log_time"].max()

('2018-04-16 00:00:00', '2018-04-30 23:59:00')

In [9]:
test["log_time"].min(), test["log_time"].max()

('2018-04-16 00:00:05', '2018-04-30 23:52:06')

In [10]:
train["country_id"].unique(), test["country_id"].unique()

(array(['xx', 'zz', 'yy'], dtype=object), array(['zz', 'yy'], dtype=object))

In [11]:
train["irank"].min(), test["irank"].min()

(1, 2)

In [12]:
train["buyer_admin_id"].nunique(), test["buyer_admin_id"].nunique()

(614960, 9844)

In [13]:
train["irank"].max(), test["irank"].max()

(264473, 39848)

In [15]:
train["buy_flag"].unique(), test["buy_flag"].unique()

(array([0, 1]), array([0, 1]))

In [16]:
train.isnull().any()

country_id        False
buyer_admin_id    False
item_id           False
log_time          False
irank             False
buy_flag          False
dtype: bool

In [17]:
test.isnull().any()

country_id        False
buyer_admin_id    False
item_id           False
log_time          False
irank             False
buy_flag          False
dtype: bool

In [20]:
train.groupby("buyer_admin_id").size().min(), test.groupby("buyer_admin_id").size().min()

(1, 1)

In [21]:
train.groupby("buyer_admin_id").size().max(), test.groupby("buyer_admin_id").size().max()

(264473, 39847)

In [22]:
pd.concat([train, test]).groupby("buyer_admin_id")["country_id"].nunique().value_counts()

1    624804
Name: country_id, dtype: int64